### Fine-tune FunctionGemma 270M for Mobile Actions

In [1]:
#export HF_TOKEN=hf_token_w_write_access #bash
#CUDA_VISIBLE_DEVICES=0 jupyter lab #bash
#Accept(ed) license on https://huggingface.co/google/functiongemma-270m-it
import os
hf_token = os.environ.get("HF_TOKEN") #from google.colab import userdata #colab only
from huggingface_hub import login
#hf_token = userdata.get('HF_TOKEN') #colab only
login(hf_token) # Login into Hugging Face Hub

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gemma_model = "google/functiongemma-270m-it"
base_model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto",
    attn_implementation="eager",
    dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

print(f"Device: {base_model.device}")
print(f"DType:  {base_model.dtype}")
print("cell finished")

Device: cuda:0
DType:  torch.bfloat16


In [3]:
import json
from random import randint
from datasets import load_dataset
from transformers import AutoTokenizer
from huggingface_hub import hf_hub_download

#/home/gy/.cache/huggingface/hub/datasets--google--mobile-actions/
#and later f models--google--functiongemma-270m-it/
data_file = hf_hub_download(repo_id="google/mobile-actions", filename="dataset.jsonl", repo_type="dataset")
dataset = load_dataset("text", data_files=data_file, encoding="utf-8")["train"].shuffle()

#print(f"\n\033[1mHere's an example from your dataset:\033[0m \n{json.dumps(json.loads(dataset[randint(0, len(dataset) - 1)]['text']), indent=2)}")
print("cell finished")

cell finished


In [4]:
import json

def apply_format(sample):
  template_iputs = json.loads(sample['text'])

  prompt_and_completion = tokenizer.apply_chat_template(
    template_iputs['messages'],
    tools=template_iputs['tools'],
    tokenize=False,
    # add_generation_prompt is False since we don't need model output after all
    # messages.
    add_generation_prompt=False)

  prompt = tokenizer.apply_chat_template(
    template_iputs['messages'][:-1],
    tools=template_iputs['tools'],
    tokenize=False,
    # add_generation_prompt is True since we would like to include
    # "<start_of_turn>model" in the prompt, if needed.
    add_generation_prompt=True)

  completion = prompt_and_completion[len(prompt):]

  return {
     "prompt": prompt,
     "completion": completion,
     "split": template_iputs["metadata"],
  }

processed_dataset = dataset.map(apply_format)
print("cell finished")

Map:   0%|          | 0/9654 [00:00<?, ? examples/s]

cell finished


In [5]:
#gue: PRINTs MOVED TO BOTTOM FOR QUICK UN/COMMENT
#@title Review the processed dataset

longest_example = max(processed_dataset, key=lambda example: len(example['prompt'] + example['completion']))
longest_example_token_count = len(tokenizer.tokenize(longest_example['prompt'] + longest_example['completion']))

max_token_count = longest_example_token_count + 100
"""
print("\033[1mHere's an example from the formatted dataset:\033[0m")
print(json.dumps(processed_dataset[randint(0, len(processed_dataset) - 1)], indent=2))

print(f"\n\033[1mThe longest example length is {len(longest_example['prompt'] + longest_example['completion'])} with {longest_example_token_count} tokens. We need to set the max_length larger than the token count in SFTConfig below.\033[0m")
print(json.dumps(longest_example, indent=2))

print(f"\n\033[1mUsing max_token_count of {max_token_count} (= {longest_example_token_count} + 100) for training below.\033[0m")
"""
print("cell finished")

cell finished


In [6]:
#@title Prepare train and eval dataset.

train_dataset = processed_dataset.filter(lambda example: example['split'] == 'train')
eval_dataset = processed_dataset.filter(lambda example: example['split'] == 'eval')
print("cell finished")

Filter:   0%|          | 0/9654 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9654 [00:00<?, ? examples/s]

In [7]:
#@title Test with a prompt

from transformers import pipeline
from random import randint
import re

# Create a transformers inference pipeline
pipe = pipeline("text-generation", model=gemma_model, tokenizer=tokenizer)

user_prompt = "Schedule a \"team meeting\" tomorrow at 4pm."  #@param {type:"string"}
messages = [
    {"role": "developer", "content": "Current date and time given in YYYY-MM-DDTHH:MM:SS format: 2024-11-15T05:59:00. You are a model that can do function calling with the following functions"},
    {"role": "user", "content": user_prompt}
]

# Reuse the tools from the sample
tools = json.loads(dataset[0]['text'])['tools']

prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False,
    add_generation_prompt=True)

print(f"\n\033[1mPrompt:\033[0m {user_prompt}")
output = pipe(prompt, max_new_tokens=max_token_count)
model_output = output[0]['generated_text'][len(prompt):].strip()

print(f"\n\033[1mBase model output:\033[0m {model_output}")
print("cell finished")

Device set to use cuda:0



Prompt: Schedule a "team meeting" tomorrow at 4pm.

Base model output: I apologize, but I cannot assist with scheduling meetings. My current tools are designed for managing specific functions like phone calls, creating contacts, opening Wi-Fi settings, and sending emails. I cannot access or manage calendar or meeting scheduling capabilities.


In [8]:
from transformers import pipeline
from random import randint
import re

# Create a transformers inference pipeline
pipe = pipeline("text-generation", model=gemma_model, tokenizer=tokenizer)

# Select a random sample from the test dataset
rand_idx = randint(0, len(train_dataset) - 1)
test_sample = train_dataset[rand_idx]

input_prompt = test_sample['prompt']
expected_output = test_sample['completion']

# Generate the output
output = pipe(input_prompt, max_new_tokens=max_token_count, skip_special_tokens=False)
actual_output = output[0]['generated_text'][len(input_prompt):].strip()

print(f"\n\033[1mInput prompt\033[0m   : {input_prompt}")
print(f"\n\033[1mExpected output\033[0m: {expected_output}")
print(f"\n\033[1mActual output\033[0m  : {actual_output}")
print("cell finished")

Device set to use cuda:0



Input prompt   : <bos><start_of_turn>developer
Current date and time given in YYYY-MM-DDTHH:MM:SS format: 2026-05-28T21:41:28
Day of week is Thursday
You are a model that can do function calling with the following functions<start_function_declaration>declaration:create_contact{description:<escape>Creates a contact in the phone's contact list.<escape>,parameters:{properties:{email:{description:<escape>The email address of the contact.<escape>,type:<escape>STRING<escape>},first_name:{description:<escape>The first name of the contact.<escape>,type:<escape>STRING<escape>},last_name:{description:<escape>The last name of the contact.<escape>,type:<escape>STRING<escape>},phone_number:{description:<escape>The phone number of the contact.<escape>,type:<escape>STRING<escape>}},required:[<escape>first_name<escape>,<escape>last_name<escape>],type:<escape>OBJECT<escape>}}<end_function_declaration><start_function_declaration>declaration:show_map{description:<escape>Shows a location on the map.<esca

In [9]:
#gue unnecessary; hard-coded below outside git-enabled dir
#from pathlib import Path
#print(Path.cwd())
#output_dir = str(Path.cwd() / "outputs" / "functiongemma-270m-mobile-actions") #STR()!
#gue end

#configure fine-tune
import torch
from transformers import AutoModelForCausalLM
from trl import SFTConfig

output_dir = "~/dl/funcgemma-mobile-actions" #gue
#output_dir = "/content/mobile-actions-functiongemma"  # Where to save your fine-tuned checkpoints
print(output_dir) #gue
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

args = SFTConfig(
    output_dir=output_dir,                            # Directory to save adapters
    num_train_epochs=2,                               # Number of training epochs
    per_device_train_batch_size=4,                    # Batch size per device during training
    gradient_accumulation_steps=8,                    # Gradient accumulation during training
    logging_strategy="steps",                         # Log every steps
    eval_strategy="steps",                            # Evaluate loss metrics based on steps
    eval_steps=50,                                    # Evaluate loss metrics every 50 steps
    logging_steps=50,                                 # Log loss metrics every 50 steps
    save_strategy="steps",                            # Save checkpoint every #gue: steps ##epoch
    save_steps=50, #gue: maybe 10 for 1st test to be able to resume
    save_total_limit=3, #gue: keep last 3
    save_safetensors=True, #gue resume-friendly
    save_on_each_node=False,
    learning_rate=1e-5,                               # Learning rate,
    lr_scheduler_type="cosine",                       # Cosine scheduler is often better for full FT
    max_length=max_token_count,                       # Max sequence length for model and packing of the dataset
    gradient_checkpointing=True,                      # Use gradient checkpointing to save memory
    packing=False,                                    # Groups multiple samples in the dataset into a single sequence
    optim="adamw_torch_fused",                        # Use fused adamw optimizer
    bf16=True,                                        # Use bf16 for mixed precision training
    completion_only_loss=True,                        # Train on completion only to improve quality
    report_to="none"                                  # No reporting.
)

base_model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto", #"none" to avoid warning model is already on multiple devices
    dtype=torch.bfloat16,
    attn_implementation='eager')

base_model.config.pad_token_id = tokenizer.pad_token_id

print("Training configured")
print("cell finished")

~/dl/funcgemma-mobile-actions
Training configured
cell finished


In [10]:
#start training
from trl import SFTTrainer

# Train and save the fine-tuned model
trainer = SFTTrainer(
    model=base_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Fine-tuned model saved to {output_dir}")

Adding EOS to train dataset:   0%|          | 0/8693 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8693 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8693 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/961 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/961 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/961 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.031600,0.021908,0.537001,1044237.000000,0.994428
100,0.015600,0.018073,0.525342,2089474.000000,0.995528
150,0.013200,0.016733,0.529755,3134537.000000,0.995891
200,0.014100,0.014855,0.541534,4177330.000000,0.996177
250,0.011800,0.014800,0.533594,5221399.000000,0.996226
300,0.012300,0.014143,0.530153,6261218.000000,0.996302
350,0.008300,0.014128,0.520362,7307784.000000,0.996314
400,0.008800,0.014170,0.520124,8349437.000000,0.996446
450,0.009400,0.014049,0.520182,9389300.000000,0.996498
500,0.009700,0.014113,0.520105,10435320.000000,0.996481


Fine-tuned model saved to ~/dl/funcgemma-mobile-actions


In [ ]:
#Plot training results
import matplotlib.pyplot as plt

# Access the log history
log_history = trainer.state.log_history

# Extract training / validation loss
train_losses = [log["loss"] for log in log_history if "loss" in log]
epoch_train = [log["epoch"] for log in log_history if "loss" in log]
eval_losses = [log["eval_loss"] for log in log_history if "eval_loss" in log]
epoch_eval = [log["epoch"] for log in log_history if "eval_loss" in log]

# Plot the training loss
plt.plot(epoch_train, train_losses, label="Training Loss")
plt.plot(epoch_eval, eval_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()
print("cell finished")

In [ ]:
#test the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Create Transformers inference pipeline
trained_model = AutoModelForCausalLM.from_pretrained(output_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(output_dir)
pipe = pipeline("text-generation", model=trained_model, tokenizer=tokenizer)
pipe_base = pipeline("text-generation", model=gemma_model, device_map="auto")

# Test a prompt
user_prompt = "Schedule a \"team meeting\" tomorrow at 4pm."  #@param {type:"string"}
messages = [
    {"role": "developer", "content": "Current date and time given in YYYY-MM-DDTHH:MM:SS format: 2024-11-15T05:59:00. You are a model that can do function calling with the following functions"},
    {"role": "user", "content": user_prompt}
]

# Reuse the tools from the sample
tools = json.loads(dataset[0]['text'])['tools']

prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False,
    add_generation_prompt=True)

print(f"\n\033[1mPrompt:\033[0m {prompt}")
output = pipe(prompt, max_new_tokens=max_token_count)
output_base = pipe_base(prompt, max_new_tokens=max_token_count)
model_output = output[0]['generated_text'][len(prompt):].strip()
model_output_base = output_base[0]['generated_text'][len(prompt):].strip()

print(f"\n\033[1mFine-tuned model output:\033[0m {model_output}")

print(f"\n\033[1mBase model output:\033[0m       {model_output_base}")
print("cell finished")

In [ ]:
#Evaluate the fine-tuned model
#@title Helper functions for evaluation

import pandas as pd

def extract_function_call(model_output):
    """
    Parses a string containing specific function call markers and returns
    a list of function call objects. Here is an example of the obejct:

    ```
    <start_function_call>call:open_map{query:<escape>San Francisco<escape>}<end_function_call>
    ```

    Args:
        model_output (str): The model output string.

    Returns:
        list: A list of dictionaries representing the function calls.
    """
    results = []

    # Pattern to extract the full content of a single function call
    # Flags: DOTALL allows matching across newlines if necessary
    call_pattern = r"<start_function_call>(.*?)<end_function_call>"
    raw_calls = re.findall(call_pattern, model_output, re.DOTALL)

    for raw_call in raw_calls:
        # Check if the content starts with 'call:'
        if not raw_call.strip().startswith("call:"):
            continue

        # Extract function name
        # Expected format: call:func_name{...}
        try:
            # Split only on the first brace to separate name and args
            pre_brace, args_segment = raw_call.split("{", 1)

            function_name = pre_brace.replace("call:", "").strip()

            # Remove the trailing closing brace '}'
            args_content = args_segment.strip()
            if args_content.endswith("}"):
                args_content = args_content[:-1]

            arguments = {}

            # Pattern to extract arguments
            # Looks for: key:<escape>value<escape>
            # The key pattern [^:,]* ensures we don't accidentally eat previous commas
            arg_pattern = r"(?P<key>[^:,]*?):<escape>(?P<value>.*?)<escape>"

            arg_matches = re.finditer(arg_pattern, args_content, re.DOTALL)

            for match in arg_matches:
                key = match.group("key").strip()
                value = match.group("value")
                arguments[key] = value

            results.append({
                "function": {
                    "name": function_name,
                    "arguments": arguments
                }
            })

        except ValueError:
            # Handles cases where syntax might be malformed (e.g., missing '{')
            continue

    return results

def extract_text(model_output):
    """
    Extracts text content and removing the <end_of_turn> marker.

    Args:
        model_output (str): The model output string.

    Returns:
        str: The cleaned text.
    """
    if not model_output or model_output.startswith("<start_function_call>"):
        return None
    return model_output.replace("<end_of_turn>", "").strip()

from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

def get_eval_logs(dataset, pipe):
  batch_size = 1
  logs = []
  # Select a random sample from the test dataset
  for i, output in enumerate(pipe(KeyDataset(dataset, "prompt"), batch_size=batch_size)):
    orig_data = dataset[i]['text']
    messages = json.loads(orig_data)['messages']
    user_message = messages[1]
    assistant_first_message = messages[2]
    input_prompt = dataset[i]['prompt']
    # Generate the output
    model_output_only = output[0]['generated_text'][len(input_prompt):].strip()

    logs.append(
        {
            # The original user prompt/query.
            "user": user_message['content'],

            # List of ground truth function call objects.
            "target_fc": assistant_first_message.get('tool_calls', []),

            # Ground truth text response.
            "target_text": assistant_first_message.get('content'),

            # List of model-generated function call objects.
            "output_fc": extract_function_call(model_output_only),

            # Model-generated text response.
            "output_text": extract_text(model_output_only),
        }
    )

    if (i + 1) % batch_size == 0:
      print(f"Eval process: {(i + 1) * 100.0 / len(dataset):.2f}%")
  return logs

def get_scored_data_frame(dataset, pipe):
  logs = get_eval_logs(dataset, pipe)
  logs_df = pd.DataFrame.from_records(logs)

  scored = pd.DataFrame()
  scored['user'] = logs_df['user']
  scored['target_names'] = logs_df['target_fc'].apply(lambda x: [fc['function']['name'] for fc in x])
  scored['output_names'] = logs_df['output_fc'].apply(lambda x: [fc['function']['name'] for fc in x])
  scored["target_arguments"] = logs_df['target_fc'].apply(lambda x: [dict(sorted(fc['function']['arguments'].items())) for fc in x])
  scored["output_arguments"] = logs_df['output_fc'].apply(lambda x: [dict(sorted(fc['function']['arguments'].items())) for fc in x])
  scored['target_text'] = logs_df['target_text']
  scored['output_text'] = logs_df['output_text']
  scored["correct_names"] = scored["target_names"] == scored["output_names"]
  scored["correct_arguments"] = scored["target_arguments"] == scored["output_arguments"]
  scored["correct"] = scored["correct_names"] & scored["correct_arguments"]

  return scored

def review(scored):
  scored["incorrect_names"] = scored["target_names"] != scored["output_names"]
  scored["incorrect_arguments"] = scored["target_arguments"] != scored["output_arguments"]
  scored["incorrect"] = scored["incorrect_names"] | scored["incorrect_arguments"]

  for index, row in scored[scored["incorrect"]].iterrows():
    print(f"\033[1mSample #{index} prompt  \033[0m: {row["user"]}")
    print(f"\033[1mSample #{index} expected\033[0m: {row["target_names"]}, {row["target_arguments"]}")
    print(f"\033[1mSample #{index} actual  \033[0m: {row["output_names"]}, {row["output_arguments"]}")
    print("---------------")

print("cell finished")

In [ ]:
#@title Evaluate the base model

base_scored = get_scored_data_frame(
    eval_dataset,
    pipeline("text-generation", model=gemma_model, device_map="auto", temperature = 0.001),
)

base_scored
print("cell finished")

In [ ]:
#@title Evaluate the fine-tuned model

from transformers import pipeline
from random import randint
import re

# Create a transformers inference pipeline
trained_model = AutoModelForCausalLM.from_pretrained(output_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(output_dir)

trained_scored = get_scored_data_frame(
    eval_dataset,
    pipeline("text-generation", model=trained_model, tokenizer=tokenizer, temperature = 0.001)
)

trained_scored
print("cell finished")

In [ ]:
#@title Compare the score of the base and fine-tuned models

# Optional: save the score in json file
trained_scored.to_json('scored_df_20251215_trained.json')
base_scored.to_json('scored_df_20251215_base.json')

print(f"\033[1mBase model score\033[0m       : {base_scored["correct"].mean()}")
print(f"\033[1mFine-tuned model score\033[0m : {trained_scored["correct"].mean()}")
print("cell finished")

In [ ]:
#Review what are not matching
review(trained_scored)
print("cell finished")

### Save your model and upload to Hugging Face Hub

In [ ]:
from huggingface_hub import ModelCard, ModelCardData, whoami

trained_model = AutoModelForCausalLM.from_pretrained(output_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(output_dir)

#@markdown Name your model
model_name = "mobile-actions"    #@param {type:"string"}

username = whoami()['name']
hf_repo_id = f"{username}/functiongemma-270m-it-{model_name}"

repo_url = trained_model.push_to_hub(hf_repo_id, create_repo=True, commit_message="Upload model")
tokenizer.push_to_hub(hf_repo_id)

card_content = f"""
---
base_model: {gemma_model}
tags:
- function-calling
- mobile-actions
- gemma
---
A fine-tuned model based on `{gemma_model}`."""
card = ModelCard(card_content)

card.push_to_hub(hf_repo_id)

print(f"Uploaded to {repo_url}")
print("cell finished")

### Conversion to .litertlm for on-device deployment
#### gue: adaption and (bug) fixes to the cell to run without above cells
#### `conda activate functiongemma2` before starting jupyter lab

In [9]:
#conda create -n functiongemma2 python=3.12 -y #for conversion to .litertlm
#conda activate functiongemma2
#install rest of functiongemma items, then mods from ipynb conversion section / part:
##pip uninstall -y tensorflow #not installed in env, maybe in colab
#!pip install ai-edge-torch-nightly --force-reinstall #install(ed) in env
##!pip install ai-edge-litert-nightly #Requirement already satisfied
#pip install sentencepiece #missing

import os
from ai_edge_torch.generative.examples.gemma3 import gemma3
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities.export_config import ExportConfig
from ai_edge_torch.generative.layers import kv_cache

#gue: moved to top and adapted to prev settings to make the conversion independent from above
checkpoint_dir = "~/dl/funcgemma-mobile-actions/checkpoint-544" #SUB to output_dir #"/content/mobile-actions-functiongemma"

litertlm_output_dir = "~/dl/funcgemma-mobile-actions-litertlm" #'/content/litertlm'
litertlm_output_dir = os.path.expanduser(litertlm_output_dir) #gue: popular expansion problem py v bash?
os.makedirs(litertlm_output_dir, exist_ok=True)

#gue 2:
output_dir = "~/dl/funcgemma-mobile-actions"
output_dir = os.path.expanduser(output_dir)
checkpoint_dir = os.path.expanduser(checkpoint_dir)

os.makedirs(checkpoint_dir, exist_ok=True)

from transformers import AutoTokenizer
import shutil

BASE_MODEL_ID = "google/functiongemma-270m-it"

tok = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=False)
spm_src = tok.vocab_file

spm_dst = os.path.join(checkpoint_dir, "tokenizer.model")
shutil.copy(spm_src, spm_dst)

assert os.path.isfile(spm_dst)

tok.save_pretrained(checkpoint_dir)
#gue end

# Metadata for FunctionGemma
llm_metadata = r"""start_token: {
    token_ids: {
        ids: [ 2 ]
    }
}
stop_tokens: {
    token_str: "<end_of_turn>"
}
stop_tokens: {
    token_str: "<start_function_response>"
}
llm_model_type: {
    function_gemma: {}
}
"""

# Create the LLM metadata file
metadata_path = os.path.join(litertlm_output_dir, 'base_llm_metadata.textproto')
with open(metadata_path, 'w') as f:
  f.write(llm_metadata)

# Import the weights and build the PyTorch model
pytorch_model = gemma3.build_model_270m(checkpoint_dir)

# Setup the export configurations and parameters for text generation models.
export_config = ExportConfig()
export_config.kvcache_layout = kv_cache.KV_LAYOUT_TRANSPOSED
export_config.mask_as_input = True

# Convert to LiteRT-LM Format
converter.convert_to_litert(
    pytorch_model,
    output_path=litertlm_output_dir,
    output_name_prefix="mobile-actions",
    prefill_seq_len=256,
    kv_cache_max_len=1024,
    quantize="dynamic_int8",
    export_config=export_config,
    #tokenizer_model_path=os.path.join(checkpoint_dir, 'tokenizer.model'),
    tokenizer_model_path=os.path.join(checkpoint_dir, "tokenizer.model"), #gue
    base_llm_metadata_path=metadata_path,
    output_format="litertlm",
)
print("cell finished")

INFO:tensorflow:Assets written to: /tmp/tmpy7o8wchy/assets


INFO:tensorflow:Assets written to: /tmp/tmpy7o8wchy/assets
W0000 00:00:1767472547.162751   44106 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1767472547.162786   44106 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1767472547.162937   44106 reader.cc:83] Reading SavedModel from: /tmp/tmpy7o8wchy
I0000 00:00:1767472547.168155   44106 reader.cc:52] Reading meta graph with tags { serve }
I0000 00:00:1767472547.168166   44106 reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpy7o8wchy
I0000 00:00:1767472547.202360   44106 loader.cc:236] Restoring SavedModel bundle.
I0000 00:00:1767472547.610956   44106 loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpy7o8wchy
I0000 00:00:1767472547.664278   44106 loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 501352 microseconds.
I0000 00:00:1767472613.944373   44106 flatbuffer_export.cc:4230] Estimated count of arithmeti

cell finished


In [ ]:
##Save the .litertlm on Google Drive #NOT!